In [1]:
import numpy as np 
import pandas as pd 
import pingouin

In [8]:
dfe = pd.read_csv('r490.csv')
dfe = dfe[dfe['r1_charge_heater'] > 0]

In [9]:
# compute spearman correlations
parameters = ['r1_temp', 'r2_temp', 'r1_pressure', 'r2_pressure']
outputs = ['process_ron', 'process_yield']
subdf = dfe[parameters + outputs]
r = subdf.corr(method='spearman')
subdf = r.loc[parameters, outputs].copy()
columns = [(o, 'Spearman') for o in outputs]
subdf.columns = columns
subdf

,"(process_ron, Spearman)","(process_yield, Spearman)"
r1_temp,-0.016607,0.031739
r2_temp,-0.332546,-0.984087
r1_pressure,-0.025440,0.018258
r2_pressure,0.001629,0.001189


In [10]:
# distance correlations
rows = []
for parameter in parameters:
    row = {
        'parameter' : parameter
    }
    for output in outputs:
        a = np.array(dfe[parameter])
        b = np.array(dfe[output])
        dcor = pingouin.distance_corr(a, b, n_boot=None)
        print("%s %s: dcor = %8.4f" % (parameter, output, dcor)) 
        row[(output, 'dCor')] = dcor
    rows.append(row)
dCorrdf = pd.DataFrame(rows).set_index('parameter')
dCorrdf

r1_temp process_ron: dcor =   0.1903
r1_temp process_yield: dcor =   0.1405
r2_temp process_ron: dcor =   0.5539
r2_temp process_yield: dcor =   0.9734
r1_pressure process_ron: dcor =   0.0215
r1_pressure process_yield: dcor =   0.0165
r2_pressure process_ron: dcor =   0.0021
r2_pressure process_yield: dcor =   0.0013


,"(process_ron, dCor)","(process_yield, dCor)"
parameter,,
r1_temp,0.190257,0.140460
r2_temp,0.553875,0.973402
r1_pressure,0.021514,0.016494
r2_pressure,0.002097,0.001265


In [11]:
# final table
corrdf = pd.concat((dCorrdf, subdf), axis=1)
corrdf.columns = pd.MultiIndex.from_tuples(corrdf.columns)
corrdf.to_csv("tmp/table1.csv")
corrdf.round(2)

,process_ron,process_yield,process_ron,process_yield
,dCor,dCor,Spearman,Spearman
r1_temp,0.19,0.14,-0.02,0.03
r2_temp,0.55,0.97,-0.33,-0.98
r1_pressure,0.02,0.02,-0.03,0.02
r2_pressure,0.00,0.00,0.00,0.00
